In [6]:
import folium
import os
import glob
import json

In [18]:
# Create map object
m = folium.Map(location=[52.51772755662563, 13.406154972878294], zoom_start=12)

# Global tooltip
tooltip_hover_marker = "Click for more info!"

# Create markers

marker_Ironhack = folium.Marker([52.532952379348814, 13.45334618603766], 
                                popup = "<strong>IRONHACK</strong>",
                                tooltip = tooltip_hover_marker).add_to(m)

maker_home = folium.Marker([52.57295915967568, 13.411971082934471],
                           popup = "<strong>Home</strong>",
                           tooltip = tooltip_hover_marker,
                           icon = folium.Icon(icon="home", color="red")).add_to(m)

# Create overlay
borough_json_files = glob.glob(r"data\geo_data\bezirksgrenzen\single_bezirk_jsons\*.geojson")
borough_list_overlay = ["Reinickendorf", "Charlottenburg-Wilmersdorf", "Treptow-Köpenick", "Pankow",
                       "Neukölln", "Lichtenberg", "Marzahn-Hellersdorf", "Spandau",
                       "Steglitz-Zehlendorf", "Mitte", "Friedrichshain-Kreuzberg", "Tempelhof-Schöneberg"]
borough_style_dict = {"fillColor": "#6290CB", "color": "#21469F"}

for idx,_ in enumerate(borough_json_files):
    borough_overlay = folium.GeoJson(borough_json_files[idx], 
                                     tooltip = borough_list_overlay[idx],
                                     name = borough_list_overlay[idx],
                                     style_function = lambda x:borough_style_dict).add_to(m)
    
# Save map
m.save("berlin_boroughs_singles.html")